<a href="https://colab.research.google.com/github/LuisRodriguez01d/TFM-RealState2022-LR/blob/Models-Optmization/TFM_MLPR_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://www.universidadviu.com/sites/universidadviu.com/themes/custom/universidadviu_com/logo.webp style="height: 100px">
<center style="color:#888">Master en Bigdata y  Data Science<br/>Trabajo Final de Master 2022</center>

## **Identificación de oportunidades de compra en el mercado inmobiliario para alquiler**

**Autor:** Luis Eduardo Rodriguez<br/>
**Módulo:** Modelado/Entrenamiento y Validacio  Alquiler

### **Descripcion:**

En el siguiente libro se busca la mejor configuracion de hyper parametros mediante gridseachCV  para MLPR



### **Librerias**

In [ ]:
#Cargamos librerias con todo lo necesario.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from sklearn.metrics import explained_variance_score

### **Carga de Datos**

Se carga la data colectada con Api idealista

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Verificar tiene la carpeta data en el drive en el path  en  mi caso es la ruta /content/drive/MyDrive/VIU/TFM/data
import os
folder = '/content/drive/MyDrive/VIU/TFM/data' # aqui el path donde estan los csv
files = os.listdir(folder)
files
#! ls /content/drive/MyDrive/VIU/TFM/data


['properties_rent_2022-03-27-19-01.csv',
 'properties_sale_2022-03-27-19-10.csv',
 'properties_rent_2022-03-27-19-16.csv',
 'properties_sale_2022-03-27-19-18.csv',
 'properties_rent_2022-04-01-17-57.csv',
 'properties_sale_2022-04-01-17-59.csv',
 'properties_rent_2022-04-01-18-05.csv',
 'properties_sale_2022-04-01-18-08.csv',
 'Idealista_search-test_insomiaReviewed.csv']

In [ ]:
#Lectura CSVs  desde directorio data desde donde esta guardado el libro, todos tiene las mismas columnas
df = pd.DataFrame()
for csv_file in Path(folder).glob('*.csv'):
    df_aux = pd.read_csv(csv_file, index_col=None, header=0)
    df = df.append(df_aux, ignore_index=True, sort=False)
#df inicial
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9391 entries, 0 to 9390
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              9391 non-null   int64  
 1   propertyCode            9391 non-null   int64  
 2   thumbnail               9328 non-null   object 
 3   externalReference       7401 non-null   object 
 4   numPhotos               9391 non-null   int64  
 5   floor                   8427 non-null   object 
 6   price                   9391 non-null   float64
 7   propertyType            9391 non-null   object 
 8   operation               9391 non-null   object 
 9   size                    9391 non-null   float64
 10  exterior                9391 non-null   bool   
 11  rooms                   9391 non-null   int64  
 12  bathrooms               9391 non-null   int64  
 13  address                 9391 non-null   object 
 14  province                9391 non-null   

### **Limpieza de Datos**

Se aplica todos los cambios  del analisis exploratorio

In [ ]:
#Eliminación de features innecesarias
dropcolumns = ['Unnamed: 0', 'thumbnail', 'externalReference', 'numPhotos', 'country',
               'hasVideo', 'newDevelopment','suggestedTexts', 'hasPlan', 'has3DTour',
               'has360', 'hasStaging', 'topNewDevelopment', 'labels', 'newDevelopmentFinished',
              'showAddress', 'priceByArea']
df = df.drop(columns=dropcolumns)

#Eliminación de duplicados
df = df.drop_duplicates(subset="url", keep='first')

#transformación de floors a números enteros y redondeamos a 1 decimal
df['floor'] = df['floor'].replace({'en': 0.5, 'bj': 0, 'ss': -0.5, 'st': -1}).astype(float)
df['floor'] = df['floor'].fillna(df['floor'].mean())
df['floor'] = df['floor'].round(decimals = 1)

#variables campos vacios con false.
df['hasLift'] = df['hasLift'].fillna(False)

#remplazamos Nulos por "{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False}"
df['parkingSpace'] = df['parkingSpace'].fillna("{'hasParkingSpace': False, 'isParkingSpaceIncludedInPrice': False}")

#Convertimos  Parking Space en: hasParkingSpace, isParkingSpaceIncludedInPrice, luego lo removemos
df_aux = df['parkingSpace'].str.split(n=4, expand=True)
df_aux.columns =['h1', 'hasParkingSpace', 'h2', 'isParkingSpaceIncludedInPrice', 'h3']
df_aux['hasParkingSpace'] = df_aux['hasParkingSpace'].replace({'False,': 'False', 'True,': 'True'})
df_aux['isParkingSpaceIncludedInPrice'] = df_aux['isParkingSpaceIncludedInPrice'].replace({'False}': 'False', 'False,': 'False', 'True}': 'True'})
df = pd.concat([df,df_aux[['hasParkingSpace', 'isParkingSpaceIncludedInPrice']]], axis=1).drop(['parkingSpace','isParkingSpaceIncludedInPrice'], axis=1)

# Sustituimos NAN en status por "good"
df['status'] = df['status'].fillna("good")

# Sustituimos NAN en distrito por municipalidades. y en vecindario por distrito
df['district'] = df['district'].fillna(df['municipality'])
df['neighborhood'] = df['neighborhood'].fillna(df['district'])

#Convertimos  detailType en los subgrupos Typology y Subtypology, remplazamos nan  de SubTypology por Typology
df_aux = df['detailedType'].str.split(n=3, expand=True)
df_aux.columns =['h1', 'typology', 'h2', 'subTypology']
df_aux['typology'] = df_aux['typology'].replace({"'chalet',": 'chalet', "'chalet'}": 'chalet',
                                                 "'flat',": 'flat', "'flat'}": 'flat',
                                                 "'countryHouse',": 'countryHouse', "'countryHouse'}": 'countryHouse'})
df_aux['subTypology'] = df_aux['subTypology'].replace({"'casaTerrera'}": 'casaTerrera', "'casale'}": 'casale',
                                                       "'countryHouse'}": 'countryHouse', "'duplex'}": 'duplex',
                                                       "'independantHouse'}": 'independantHouse', "'studio'}": 'studio',
                                                      "'terracedHouse'}": 'terracedHouse', "'penthouse'}": 'penthouse',
                                                      "'semidetachedHouse'}": 'semidetachedHouse'})
df_aux['subTypology'] = df_aux['subTypology'].fillna(df_aux['typology'])

df = pd.concat([df,df_aux[['typology', 'subTypology']]], axis=1).drop(['detailedType','typology'], axis=1)

#Transformamos en variables numéricas  todas  las tipo binario (True or False)
columns = df.loc[: , (df == 'False').any()].columns
for column in columns:
    df[column] = df[column].map({'True': True, 'False': False})
columns = df.select_dtypes('bool').columns
for column in columns:
    df[column] = df[column].astype(float)

#Filtrado de Muestras de barcelona  distance < 5000
df = df[((df['province']=="Barcelona") & (df['distance']<5000)) | (df['province']=="Madrid")]

#Insersion de variable logprice
df['Logprice'] = np.log(df.price.values)

#Quitamos Outlier, para viviendas en venta y alquiler valores obtenidos del analisis exploratorio:
cleaned_df = df[((df['operation']=="rent") & (df['price']<4730)) | (df['operation']=="sale")]
cleaned_df = cleaned_df[((cleaned_df['operation']=="sale") & (cleaned_df['price']<1691000)) | (cleaned_df['operation']=="rent")]

#Quitamos Outlier en el tamaño, en este caso pisos mayores a 296m2
cleaned_df = cleaned_df[cleaned_df['size'] < 296]
#Quitamos Outliers, que vemos que son pisos con más de 9 cuartos
cleaned_df = cleaned_df[cleaned_df['rooms'] <= 9]
#Quitamos Outliers, que vemos que son pisos con más de 5 baños
cleaned_df = cleaned_df[cleaned_df['bathrooms'] <= 5]
#Quitamos Outliers, que vemos que son pisos con más de 13 pisos
cleaned_df = cleaned_df[cleaned_df['floor'] <= 13]

#df  final despues de la limpieza
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7620 entries, 0 to 9389
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   propertyCode     7620 non-null   int64  
 1   floor            7620 non-null   float64
 2   price            7620 non-null   float64
 3   propertyType     7620 non-null   object 
 4   operation        7620 non-null   object 
 5   size             7620 non-null   float64
 6   exterior         7620 non-null   float64
 7   rooms            7620 non-null   int64  
 8   bathrooms        7620 non-null   int64  
 9   address          7620 non-null   object 
 10  province         7620 non-null   object 
 11  municipality     7620 non-null   object 
 12  district         7620 non-null   object 
 13  neighborhood     7620 non-null   object 
 14  latitude         7620 non-null   float64
 15  longitude        7620 non-null   float64
 16  url              7620 non-null   object 
 17  distance      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
cleaned_df.columns

Index(['propertyCode', 'floor', 'price', 'propertyType', 'operation', 'size',
       'exterior', 'rooms', 'bathrooms', 'address', 'province', 'municipality',
       'district', 'neighborhood', 'latitude', 'longitude', 'url', 'distance',
       'description', 'status', 'hasLift', 'hasParkingSpace', 'subTypology',
       'Logprice'],
      dtype='object')

### **Separamos Datos de Ventas y Alquiler**

In [ ]:
#separamos el dataset por los dos tipos de operación y borramos la etiqueta, para este modulo solo trabajos con alquiler

df_rent = cleaned_df[cleaned_df['operation']== 'rent']
df_rent = df_rent.drop('operation', axis=1)

df_sale = cleaned_df[cleaned_df['operation']== 'sale']
df_sale = df_sale.drop('operation', axis=1)

### Datos Sinteticos
Creamos una variable ProvMunDist concat(Province-Municipality-Distric)   para evitar duplicidades
Calculamos el precio medio del distrito, luego decidimos cual usar

In [ ]:
#Creación de Variables sintéticas para agrupar Province|Municipality|District|Neighbourhood

df_rent['pmdistrict'] = df_rent['province'] + '|' + df_rent['municipality'] + '|' +  df_rent['district']
df_sale['pmdistrict'] = df_sale['province'] + '|' + df_sale['municipality'] + '|' +  df_sale['district']

df_rent['pmdneigh'] = df_rent['province'] + '|' + df_rent['municipality'] + '|' +  df_rent['district'] + '|' +  df_rent['neighborhood']
df_sale['pmdneigh'] = df_sale['province'] + '|' + df_sale['municipality'] + '|' +  df_sale['district'] + '|' +  df_sale['neighborhood']

In [ ]:
#Creación de variable sintética mean price por provincia y distrito

d_mean = df_rent.groupby('pmdistrict').mean()
d_mean = d_mean.rename(columns={"price": "priced_mean"})

n_mean = df_rent.groupby('pmdneigh').mean()
n_mean = n_mean.rename(columns={"price": "pricen_mean"})

df_rent = pd.merge(df_rent, d_mean['priced_mean'], how='left', left_on='pmdistrict', right_on=d_mean.index)
df_rent = pd.merge(df_rent, n_mean['pricen_mean'], how='left', left_on='pmdneigh', right_on=n_mean.index)

### Regresion Preparation

In [ ]:
# Seleccionamos features,  trabajos con logprice
features=['floor', 'Logprice', 'size', 'exterior', 'rooms', 'bathrooms', 'distance', 
          'hasLift', 'hasParkingSpace', 'propertyType', 'pmdistrict']
rent_df_regresion = df_rent[features]
#sale_df_regresion = df_rent.copy()

#Convertimos estas variables a dummies 
rent_df_regresion = pd.get_dummies(rent_df_regresion, columns=['pmdistrict'])
rent_df_regresion = pd.get_dummies(rent_df_regresion, columns=['propertyType'])
#convertimos todos estos campos a ENTEROS y rellenamos con False los nulos.

rent_df_regresion['bathrooms'] = rent_df_regresion['bathrooms'].astype(float)
rent_df_regresion['rooms'] = rent_df_regresion['rooms'].astype(float)
rent_df_regresion['distance'] = rent_df_regresion['distance'].astype(float)

#Revisamos el DF preparado

rent_df_regresion.info(verbose=True, show_counts=True)
rent_df_regresion.columns
rent_df_regresion.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4236 entries, 0 to 4235
Data columns (total 139 columns):
 #    Column                                                                     Non-Null Count  Dtype  
---   ------                                                                     --------------  -----  
 0    floor                                                                      4236 non-null   float64
 1    Logprice                                                                   4236 non-null   float64
 2    size                                                                       4236 non-null   float64
 3    exterior                                                                   4236 non-null   float64
 4    rooms                                                                      4236 non-null   float64
 5    bathrooms                                                                  4236 non-null   float64
 6    distance                                      

,floor,Logprice,size,exterior,rooms,bathrooms,distance,hasLift,hasParkingSpace,pmdistrict_Barcelona|Barcelona|Ciutat Vella,...,pmdistrict_Madrid|Tres Cantos|Segunda Fase,pmdistrict_Madrid|Velilla de San Antonio|Velilla de San Antonio,pmdistrict_Madrid|Villaviciosa de Odón|Centro,pmdistrict_Madrid|Villaviciosa de Odón|El Bosque,propertyType_chalet,propertyType_countryHouse,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio
0,4.0,7.003065,92.0,1.0,3.0,2.0,9352.0,1.0,1.0,0,...,0,0,0,0,0,0,0,1,0,0
1,2.0,7.003065,124.0,1.0,3.0,2.0,7616.0,1.0,1.0,0,...,0,0,0,0,0,0,0,1,0,0
2,2.0,6.461468,59.0,1.0,1.0,1.0,15502.0,1.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,1.0,7.554335,110.0,0.0,3.0,2.0,3167.0,1.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
4,3.0,6.565265,60.0,0.0,2.0,1.0,7993.0,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0


### Separamos df en entrenamiento  y test (30%)

In [ ]:
from sklearn.model_selection import train_test_split

#En el dataset Y incluimos los datos a predecir, que es el rango de precios
df_y = rent_df_regresion['Logprice']
dataset_y = np.array(df_y)

#En el dataset X borramos la varible a predecir
dataset_X = rent_df_regresion.drop('Logprice', axis=1)

#Usamos RobustScaler
from sklearn.preprocessing import RobustScaler
rent_df_regresion_robust = RobustScaler().fit_transform(dataset_X)

#Generamos los datos de Entrenamiento y Test, al 70 - 30
X_train, X_test, y_train, y_test = train_test_split(rent_df_regresion_robust, dataset_y, test_size=.3, random_state=42)

print(len(X_train))
print(len(X_test))

2965
1271


### Preparamos el entorno

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.metrics import precision_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import SCORERS

from sklearn.neural_network import MLPRegressor




### GradientBoostingRegressor

In [ ]:
mlpr = MLPRegressor(random_state=42)
mlpr.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [ ]:
# Buscamos los mejores HiperParametros para nuestro caso.

param_grid={'hidden_layer_sizes': [(100,),(200,)], #MLPR
            'learning_rate': ['constant','adaptive'],#MLPR
            'solver': ['sgd', 'adam'], #MLPR
            'max_iter': [100,200,500,1000],#MLPR
            'alpha': [0.0001,0.001,0.01], 
            'activation': ['relu','tanh','logistic']#MLPR
           } 
    
gbr_grid = GridSearchCV(mlpr,
                        param_grid,
                        cv = 2,
                        n_jobs = -1,
                        verbose=True)

In [ ]:
grid_result_bgr = gbr_grid.fit(X_train, y_train)

best_params = grid_result_bgr.best_params_
best_params

Fitting 2 folds for each of 288 candidates, totalling 576 fits


{'activation': 'logistic',
 'alpha': 0.01,
 'hidden_layer_sizes': (200,),
 'learning_rate': 'constant',
 'max_iter': 100,
 'solver': 'adam'}